In [1]:
import pandas as pd
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt

In [2]:
low_memory=False
df = pd.read_csv('MyFoodData-Nutrition-Facts-SpreadSheet-Release-1-4.csv')
df.name = df.name.apply(lambda x: x[:18])#shortens item names to 18 characters

#selects the columns for the graph plot (first 25 excluding water)
df2 = df[df.columns[1:26]]
df3 = df2[df2.columns[0:25]]

#calculating the %DV for each nutrient
df3['Calories'] = df2['Calories']/2000 * 100
df3['Fat (g)'] = df2['Fat (g)']/78 * 100
df3['Protein (g)'] = df2['Protein (g)']/50 * 100
df3['Carbohydrate (g)'] = df2['Carbohydrate (g)']/300 * 100 
df3['Sugars (g)'] = df2['Sugars (g)']/50 * 100
df3['Fiber (g)'] = df2['Fiber (g)']/28 * 100
df3['Cholesterol (mg)'] = df2['Cholesterol (mg)']/300 * 100
df3['Saturated Fats (g)'] = df2['Saturated Fats (g)']/20 * 100
df3['Calcium (mg)'] = df2['Calcium (mg)']/1300 * 100
df3['Iron, Fe (mg)'] = df2['Iron, Fe (mg)']/18 * 100
df3['Potassium, K (mg)'] = df2['Potassium, K (mg)']/4700 * 100
df3['Magnesium (mg)'] = df2['Magnesium (mg)']/420 * 100
df3['Vitamin A, IU (IU)'] = df2['Vitamin A, IU (IU)']/3000 * 100
df3['Vitamin A, RAE (mcg)'] = df2['Vitamin A, RAE (mcg)']/900 * 100
df3['Vitamin C (mg)'] = df2['Vitamin C (mg)']/90 * 100
df3['Vitamin B-12 (mcg)'] = df2['Vitamin B-12 (mcg)']/2.4 * 100
df3['Vitamin D (mcg)'] = df2['Vitamin D (mcg)']/20 * 100
df3['Vitamin E (Alpha-Tocopherol) (mg)'] = df2['Vitamin E (Alpha-Tocopherol) (mg)']/15 * 100
df3.rename(columns = {'Vitamin E (Alpha-Tocopherol) (mg)':'Vitamin E (mg)'}, inplace = True)
df3['Added Sugar (g)'] = df2['Added Sugar (g)']/50 * 100
#df3['Net-Carbs (g)'] = df2['Net-Carbs (g)']/ * 100
#df3['Water (g)'] = df2['Water (g)']/ * 100
df3['Omega 3s (mg)'] = df2['Omega 3s (mg)']/1600 * 100 #%AdequateIntake
df3['Omega 6s (mg)'] = df2['Omega 6s (mg)']/1200 * 100 #%AI for women

In [ ]:
# -*- encoding: utf-8 -*-


#Class Definitions and Builder

from kivy.config import Config
Config.set('graphics', 'fullscreen', '0')
from kivy.core.window import Window
from kivy.garden.matplotlib.backend_kivyagg import FigureCanvasKivyAgg
from kivy.properties import ListProperty, StringProperty, NumericProperty, ObjectProperty
from kivy.uix.widget import Widget
from kivy.uix.button import Button
from kivy.uix.dropdown import DropDown
from kivy.uix.textinput import TextInput
from kivy.lang import Builder
import pandas as pd
import numpy as np

Window.size=(700,700)
Window.clearcolor = (.95, .95, .95, 1)

Builder.load_string('''
<Button>
    size_hint:(.3, None)
    #width:100
    height:30

#<Chooser@TextInput>:
    #id: food
    #hint_text:'Enter a Food Item'
    #size_hint:(0.3,None)
    #pos_hint:{'x':0.6, 'top':.97}
    #height:30
    #width:100
    #background_normal: ''
    #background_color: (1,1,1,1)
    
<Label>
    size_hint:(0.3,None)
    height:30
    
<MyLayout>
    top_layout: topLayoutID
    graph:graph
    food:food
        
    FloatLayout:
        id: topLayoutID
        size: root.width, root.height
        
        Label:
            text: "Food:"
            background_color:(0, 0, 0, 1)
            pos_hint: {'x':0.05, 'top':.955}
            canvas.before:
                Color:
                    rgba: self.background_color
                Rectangle:
                    size: self.size
                    pos: self.pos
        Chooser:
            id: food
            hint_text:'Enter a Food Item'
            size_hint:(0.3,None)
            pos_hint:{'x':0.35, 'top':.955}
            height:30
            width:100
            background_normal: ''
            background_color: (1,1,1,1)
            
        Button:
            text:'Confirm'
            color: 1,1,1,1
            background_normal:''
            background_color:(220/255, 20/255, 60/255, 1)
            pos_hint:{'x':0.65, 'top':.955}
            on_press:root.run_functions()
            
    GridLayout:
        id: graph
        cols:1
        size: root.width, root.height
        ''')

class MyLayout(Widget):
    df = pd.read_csv('MyFoodData-Nutrition-Facts-SpreadSheet-Release-1-4.csv')
    df.name = df.name.apply(lambda x: x[:18])
    data = df.sort_values(by=['name'])
    choices = data.name
    
    i = NumericProperty(0)
    first_plot = NumericProperty(0)
    userinput = ListProperty([None]*20)
    xtext = StringProperty('')
    ytext = StringProperty('')
    ztext = StringProperty('')
    
    def clear(self):
        self.ids.food.text = ''
            
    def graph_item(self):
        self.userinput[self.i] = self.ids.food.text
        if self.i== 0:
            self.xtext = "(data.index==\"{}\")".format(self.userinput[self.i])
            self.ztext = self.xtext
        else:
            self.ytext = "|(data.index==\"{}\")".format(self.userinput[self.i])
            self.ztext = self.xtext + self.ytext
            self.xtext = self.ztext

        self.i += 1

        #matplot graphing
        mpl.rc('xtick', labelsize=10)

        data = df3.round(2)
        data = data.sort_values(by=['name'])
        data = data.set_index('name')

        data = data[eval(self.ztext)]#|(data.name=='Bananas')]
        group1 = list(data.columns[1:21])
        group2 = list(data.columns[22:24])
        new_cols = group1+group2
        set(data.columns)==set(new_cols)
        data = data[new_cols]

        data = data.T
        colors = ["#f94144","#f3722c","#f8961e","#f9c74f","#90be6d","#43aa8b","#577590"]


        l = self.userinput[:self.i] #convert userinputs into list
        l.sort() #sort in ascending order
        convertL = ' '.join([str(e) for e in l]) #join elements of l into a string with spaces between each element
        new_convertL = convertL.replace(" ", ", ")

        if self.first_plot > 0:
            plt.cla()
            self.graph.clear_widgets()

        ax = data.plot(kind='bar', title='Total %DV for {}'.format(new_convertL), width=0.8, figsize=(5,5), color=colors, fontsize=12)

        #graph labels
        for rect in ax.patches:
            # Find where everything is located
            height = rect.get_height()
            width = rect.get_width()
            x = rect.get_x()
            y = rect.get_y()

            # The height of the bar is the data value and can be used as the label
            label_text = f'{height:.2f}'  # f'{height:.2f}' to format decimal values

            # ax.text(x, y, text)
            label_x = x + width / 2
            label_y = y + height / 2

            # plot only when height is greater than specified value
            if height >= 1.5:
                ax.text(label_x, label_y, label_text, ha='center', va='center', fontsize=8, fontweight='bold')

        plt.legend(l)
        plt.ylabel('%DV of Total Recommended', fontsize=15, fontweight='bold')
        plt.xlabel('Nutrients', fontsize=15, fontweight='bold')
        ax.set_ylim(0,120)
        plt.axhline(100, color="gray", lw=2, ls='--')
        
    def display_graph(self):
        self.graph.add_widget(FigureCanvasKivyAgg(plt.gcf()))        
        self.first_plot +=1
        
    def run_functions(self):
        if self.ids.food.text == '':
            pass
        
        elif self.ids.food.text.startswith(' '):
            self.clear()
            
        elif self.ids.food.text not in self.ids.food.choiceslist:
            self.ids.food.hint_text='Item Not Found. Please Try Again'
            self.clear()
            
        elif self.ids.food.text in self.userinput:
            self.ids.food.hint_text='Item has already been entered'
            self.clear()
            
        else:
            self.graph_item()
            self.display_graph()
            self.clear()
    
    
class Chooser(TextInput):
    choiceslist = ListProperty([])

    def __init__(self, **kwargs):
        df = pd.read_csv('MyFoodData-Nutrition-Facts-SpreadSheet-Release-1-4.csv')
        df.name = df.name.apply(lambda x: x[:18])
        data = df.sort_values(by=['name'])
        choices = data.name
        
        self.choiceslist = choices  # list of choices
        super(Chooser, self).__init__(**kwargs)
        self.multiline = False
        self.halign = 'left'
        self.bind(text=self.on_text)
        self.dropdown = None
        
    def open_dropdown(self, *args):
        if self.dropdown:
            self.dropdown.open(self)

    def keyboard_on_key_down(self, window, keycode, text, modifiers):
        if self.suggestion_text and keycode[0] == ord('\r'):  # enter selects current suggestion
            self.suggestion_text = ' '  # setting suggestion_text to '' screws everything
            self.text = self.values[0]
                        
            if self.dropdown:
                self.dropdown.dismiss()
                self.dropdown = None
        else:
            super(Chooser, self).keyboard_on_key_down(window, keycode, text, modifiers)

    def on_text(self, chooser, text):
        if self.dropdown:
            self.dropdown.dismiss()
            self.dropdown = None
        if text == '':
            return
        values = []
        
        for addr in self.choiceslist:#for each item in choicelist, check if text matches if text>2
            newaddr = addr.lower()#convert to lowercase
            if len(self.text)> 2:
                if newaddr.startswith(text.lower()) and addr not in values:
                    values.append(addr)
                    values = sorted(values, key=len)#sort by length, ascending order
                    values = values[:5] 
                    
        for addr in self.choiceslist:
            if len(self.text)> 2:
                newaddr = addr.lower()
                if text.lower() in newaddr and addr not in values and len(values) < 5:
                        values.append(addr)
                        values = sorted(values, key=len)
                        values = values[:5]
                        
        self.values = values
        if len(values) > 0:
            if len(self.text) < len(self.values[0]):
                self.suggestion_text = self.values[0][len(self.text):]
            else:
                self.suggestion_text = ' '  # setting suggestion_text to '' screws everything
            self.dropdown = DropDown()
            for val in self.values:
                btn = Button(
                    text=val, 
                    size_hint_y=None, 
                    height=30, 
                    color= (0,0,0,1), 
                    background_normal= '',
                    background_color= (1,1,1,1),
                    on_release=self.do_choose)
                
                self.dropdown.add_widget(btn)
            self.dropdown.open(self)

    def do_choose(self, butt):
        self.text = butt.text
        if self.dropdown:
            self.dropdown.dismiss()
            self.dropdown = None
    
if __name__ == '__main__':
    from kivy.app import App
    from kivy.uix.relativelayout import RelativeLayout

    class TestApp(App):
        top_layout = ObjectProperty(None)
        def build(self):
            return MyLayout()


    TestApp().run()